In [23]:
from typing import Tuple
from OCP.gp import gp_Pnt
from OCP.BRepMesh import BRepMesh_IncrementalMesh
from OCP.BRepBuilderAPI import BRepBuilderAPI_MakeVertex
from OCP.TopExp import TopExp_Explorer
from OCP.TopoDS import TopoDS, TopoDS_Shape, TopoDS_Edge, TopoDS_Wire, TopoDS_Face, TopoDS_Vertex
from OCP.TopAbs import TopAbs_WIRE, TopAbs_EDGE, TopAbs_REVERSED, TopAbs_INTERNAL
from OCP.BRepMesh import BRepMesh_IncrementalMesh
from OCP.BRep import BRep_Tool
from OCP.TopTools import TopTools_IndexedMapOfShape
from OCP.TopLoc import TopLoc_Location
import cadquery as cq
from cadquery.func import *
from cadquery.selectors import *

class FeatureExtract(Selector):
    def __init__(self, feature):
        self.feature = feature

    def filter(self, _):
        return [self.feature]

Triangles = List[Tuple[int, int, int]]

class WorkPlaneImpl:
    def __init__(self, obj, tolerance: float = 0.1, angular_tolerance: float = 0.1):
        self.obj = obj
        self.vertice_index_map = TopTools_IndexedMapOfShape()
        self.edge_index_map = TopTools_IndexedMapOfShape()
        self.faces: dict = {}
        self.edges: dict = {} # edges uuid -> [edge1, edge2, ...], edge = [pointi, pointj, ...]
        self.wires: dict = {} # wires uuid -> [wire1, wire2, ...], wire = [edgei, edgej, ...]
        self.vertices: dict = {}
        self._triangulation(tolerance, angular_tolerance)

    def _insert_vertex(self, pnt, tag=None):
        vertex = BRepBuilderAPI_MakeVertex(pnt).Vertex() if isinstance(pnt, gp_Pnt) else pnt
        if (not self.vertice_index_map.Contains(vertex)):
            self.vertice_index_map.Add(vertex)
            if (tag != None):
                self.obj.vertices(FeatureExtract(vertex)).tag(tag)
                self.vertices[tag] = self.vertice_index_map.FindIndex(vertex) - 1
        return self.vertice_index_map.FindIndex(vertex) - 1 if tag == None else self.vertices[tag]

    def _find_vertex(self, vertex):
        return self.vertice_index_map.FindIndex(vertex) - 1
    
    def _insert_edge(self, edge: TopoDS_Edge, vertices_in_edge):
        if (not self.edge_index_map.Contains(edge)):
            self.edge_index_map.Add(edge)
            tag = f"edge-{len(self.edges)}"
            self.obj.edges(FeatureExtract(edge)).tag(tag)
            self.edges[tag] = vertices_in_edge
        return self.edge_index_map.FindIndex(edge) - 1
    
    def _traverseWire(self, wire_ex: any, bt, triangulation, location):
        while wire_ex.More():
            wire = TopoDS.Wire_s(wire_ex.Current())
            if not wire:
                wire_ex.Next()
                continue
            edge_ex = TopExp_Explorer(wire, TopAbs_EDGE)
            edges_in_wire = self._traverseEdge(edge_ex, bt, triangulation, location)
            tag = f"wire-{len(self.wires)}"
            self.obj.wires(FeatureExtract(wire)).tag(tag)
            self.wires[tag] = edges_in_wire
            wire_ex.Next()

    def _traverseEdge(self, edge_ex: any, bt, triangulation, location):
        edges_in_wire = []
        while edge_ex.More():
            vertices_in_edge = []
            edge = TopoDS.Edge_s(edge_ex.Current())
            if not edge:
                edge_ex.Next()
                continue
            poly = bt.PolygonOnTriangulation_s(edge, triangulation, location)
            if not poly:
                edge_ex.Next()
                continue
            nb_edge = poly.NbNodes()
            for i in range(1, nb_edge + 1):
                pnt = triangulation.Node(poly.Node(i)).Transformed(location.Transformation())
                vertices_in_edge.append(self._insert_vertex(pnt))
            edges_in_wire.append(self._insert_edge(edge, vertices_in_edge))
            edge_ex.Next()
        return edges_in_wire

    def _parse_face(self, face, triangulation, location):
        nb_nodes = triangulation.NbNodes()
        nb_triangles = triangulation.NbTriangles()
        local_index = {}
        for i in range(1, nb_nodes + 1):
            pnt = triangulation.Node(i).Transformed(location.Transformation())
            local_index[i] = self._insert_vertex(pnt)
        triangles = []
        for i in range(1, nb_triangles + 1):
            i1, i2, i3 = triangulation.Triangle(i).Get()
            if face.wrapped.Orientation() in (TopAbs_REVERSED, TopAbs_INTERNAL):
                i1, i2 = i2, i1
            ind1 = local_index[i1]
            ind2 = local_index[i2]
            ind3 = local_index[i3]
            triangles.append((ind1, ind2, ind3))
        tag = f"face-{len(self.faces)}"
        self.obj.faces(FeatureExtract(face)).tag(tag)
        self.faces[tag] = triangles
    
    def _triangulation(self, tolerance: float, angular_tolerance: float):
        shape = self.obj.val().wrapped
        BRepMesh_IncrementalMesh(shape, tolerance, True, angular_tolerance, True)
        bt = BRep_Tool()
        for face in self.obj.faces():
            location = TopLoc_Location()
            triangulation = bt.Triangulation_s(face.wrapped, location)
            if not triangulation:
                continue
            wire_ex = TopExp_Explorer(face.wrapped, TopAbs_WIRE)
            self._traverseWire(wire_ex, bt, triangulation, location)
            self._parse_face(face, triangulation, location)
        
        for vertex in self.obj.vertices():
            tag = f"vertex-{len(self.vertices)}"
            self.vertices[tag] = self._insert_vertex(vertex.wrapped)
    
    @property
    def MeshPoints(self):
        points = []
        for i in range(1, self.vertice_index_map.Extent() + 1):
            vertex = TopoDS.Vertex_s(self.vertice_index_map.FindKey(i))
            pnt = BRep_Tool.Pnt_s(vertex)
            points.append((pnt.X(), pnt.Y(), pnt.Z()))
        return points
    @property
    def Vertices(self):
        return self.vertices
    @property
    def Edges(self):
        return self.edges
    @property
    def Wires(self):
        return self.wires
    @property
    def Faces(self):
        return self.faces
    def GetFeature(self, type: str,tag: str):
        return self.obj

box = cq.Workplane("XZ").circle(2.0).extrude(100)
object_box = WorkPlaneImpl(box, 0.1, 0.1)
# for i in object_box.Vertices.values():
#     print(object_box.MeshPoints[i])
print(object_box.Faces.keys())

dict_keys(['face-0', 'face-1', 'face-2'])


In [39]:
new_object = object_box.obj.faces(None, 'face-1').workplane().hole(0.11)
new_object = new_object.faces(None, 'face-2').circle(3.0).extrude(10)
new_object.exportSvg("./test.svg")

In [ ]:
import ast
import types

class CodeExecutor:
    def __init__(self):
        self.workplanes = {}
        self.sketchs = {}

    def LoadFromSource(self, source_code: str):
        tree = ast.parse(source_code)
        

def run_workplanes_from_source(source_code: str):
    tree = ast.parse(source_code)
    target_func_names = []
    for node in tree.body:
        if isinstance(node, ast.FunctionDef):
            for deco in node.decorator_list:
                if isinstance(deco, ast.Name) and deco.id == 'workplane':
                    target_func_names.append(node.name)

    def workplane(fn): return fn  
    mod = types.ModuleType("mod")
    mod.__dict__['workplane'] = workplane

    # Step 3: 執行整個 source code
    exec(source_code, mod.__dict__)

    # Step 4: 執行標記過的函數，收集結果
    result_dict = {}
    for name in target_func_names:
        func = getattr(mod, name, None)
        if callable(func):
            result_dict[name] = func()
    return result_dict


In [40]:
source = '''
@workplane
def cube():
    return "cube_model"

@workplane
def cyl():
    return "cylinder_model"

def not_this_one():
    return "nope"
'''

results = run_workplanes_from_source(source)
print(results)
# 👉 {'cube': 'cube_model', 'cyl': 'cylinder_model'}


{'cube': 'cube_model', 'cyl': 'cylinder_model'}
